In [1]:
#! pip3 install xlrd >= 1.0.0
#! pip3 install openpyxl
#! pip3 install pattern

In [9]:
#load module
import pandas as pd
import re
import nltk
import codecs
import string
from pattern.en import parsetree
from pattern.en import parse
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
#load data
data = pd.read_excel('Unstructured Data English.xlsx', encoding='utf-8', index_col=0)
data.head()

,Katakana text Translated,Unnamed: 2,Unnamed: 3
ID,,,
1,Huge @Patriots pep rally at Toso's in PHX...wa...,NaN,NaN
2,For Robert Kraft and Roger Goodell air still i...,NaN,NaN
3,For Robert Kraft and Roger Goodell air still i...,NaN,NaN
4,Need #recipes? Visit the Kraft #CookingUpGood ...,NaN,NaN
5,@EdgeofSports You take the good-You take the b...,NaN,NaN


In [4]:
data.columns
data.drop(['Unnamed: 2','Unnamed: 3'],axis=1,inplace=True)
data.columns

Index(['Katakana text Translated'], dtype='object')

In [5]:
data.rename(columns={'Katakana text Translated': 'text'}, inplace=True)
data.head(5)

,text
ID,
1,Huge @Patriots pep rally at Toso's in PHX...wa...
2,For Robert Kraft and Roger Goodell air still i...
3,For Robert Kraft and Roger Goodell air still i...
4,Need #recipes? Visit the Kraft #CookingUpGood ...
5,@EdgeofSports You take the good-You take the b...


In [6]:
#analyse overview of data

#### Reference Data , that needs to be taken care.
    Kraft Canada: New Printable Coupons Available! Save On Shake N ’ Bake MiO and Jell-O
    Have you heard the latest about the Patriots? Bob Kraft paid the attendant to deflate the balls...in Singles. #thispunischeesy #onlykidding
    @shalisemyoung @jmarsh4037 ask him if he'd rather hand the SB trophy to Marshawn/Sherman or Kraft/Brady #awkward

    RT @brijo20: The Pikes are seriously so sweet <ed><U+00A0><U+00BD><ed><U+00B8><U+008D>

    500 #0 6.5x10 KRAFT BUBBLE MAILERS PADDED MAILING ENVELOPE BAGS (Imperfect) http://t.co/mlR90zaEIR http://t.co/QqPXVOdQE9

    kraft's macaroni &amp; cheese &gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt; any other brand


#### Observations:
    data has 
    1. http urls, that needs to be removed.
    2. theme tags needs to be stripped to remove "#"
    3. similarly user tags needs to be stripped to remove "@"
    4. &amp; is used. which used to write “&” symbol in HTML. this needs to be replaced.

In [10]:
#cleanse data

emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]

# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

#Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
stop_words = nltk.corpus.stopwords.words('english')
stop_words.extend(['rt','via',"isn't","ain't","don't","hasn't","haven't","I've","I'VE","i've",'mr','mrs','Mr','Mrs','MR','MRS'])
stop_words.extend([u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u'not', u'only', u'own', u'same', u'so', u'than', u'too', u'very', u's', u't', u'can', u'will', u'just', u'don', u'should', u'now'])
stop_words = set(stop_words)
wn = nltk.WordNetLemmatizer()

def get_phrases(sentence):
    phrase_dict = {}
    parse_tree = parsetree(sentence, relations=True, lemmata=True)
    for sent in parse_tree:
        for chunk in sent.chunks:
            phrase_key = chunk.type
            phrase_val = ' '.join([w.string for w in chunk.words])

            if not phrase_dict.get(phrase_key, []):
                phrase_dict[phrase_key] = []
            if len(phrase_val.split())>1:
                phrase_dict[phrase_key].append(phrase_val)

    return phrase_dict
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=True):
    
    # substitute hyper link with empty string
    s = re.sub(r'http\S+', '', s)
    
    # substitute html tabs with empty string
    s = re.sub(r'<.*?>', '', s)
    
    #remove numbers from tweet
    s = re.sub(r'\w*\d\w*', '', s)
    s = re.sub(r'(?:(?:\d+,?)+(?:\.?\d+)?)','', s)
    
    #remove &amp; from tweet
    s = s.replace('&amp;','&')
    s = s.replace('’',"'")
    s = s.replace('-'," ")
    
    phrases = []
    phrases = get_phrases(s).get('NP',[])
    if lowercase:
        phrases = [word_p.lower() for word_p in phrases]
    
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    
    #check tokens against stop words , emoticons and punctuations
    tokens = [word.replace("'s",'') for word in tokens if word not in stop_words and word not in emoticons and word not in string.punctuation]
    
    #remove work with one or two chracter length.
    tokens = [word for word in tokens if len(word)>2]
    
    tokens = [wn.lemmatize(word) for word in tokens]
    
    return (phrases, tokens)
 
tweet = "RT @marcobonzanini: just &amp; <abcd> an example! :D http://example.com #NLP 123 12/45/2020 987654321 SD09875 sam's"
print(preprocess(tweet))
# ['RT', '@marcobonzanini', ':', 'just', 'an', 'example', '!', ':D', 'http://example.com', '#NLP']

(['rt @marcobonzanini', 'an example', 'nlp // sam'], ['@marcobonzanini', 'example', '#nlp', 'sam'])


In [11]:
data.head()

,text
ID,
1,Huge @Patriots pep rally at Toso's in PHX...wa...
2,For Robert Kraft and Roger Goodell air still i...
3,For Robert Kraft and Roger Goodell air still i...
4,Need #recipes? Visit the Kraft #CookingUpGood ...
5,@EdgeofSports You take the good-You take the b...


In [13]:
#data['cleansed_text'] = data['text'].apply(lambda x: preprocess(x))
data[['phrases', 'cleansed_tokens']] = data.apply(lambda row: pd.Series(preprocess(row['text'])), axis=1)

In [14]:
for rec in data[:15].iterrows():
    print(rec[1]['text']," --- ",rec[1]['cleansed_tokens']," --- ",rec[1]['phrases'])
    print()

Huge @Patriots pep rally at Toso's in PHX...waiting for Mr Kraft's arrival #GoPats @PatriotsExtra http://t.co/y9NElMN6jo  ---  ['huge', '@patriots', 'pep', 'rally', 'toso', 'phx', 'waiting', 'kraft', 'arrival', '#gopats', '@patriotsextra']  ---  ['huge @patriots pep rally', 'mr kraft']

For Robert Kraft and Roger Goodell air still isn't clear - Boston Globe http://t.co/Rw4oh3hswK #google #sport  ---  ['robert', 'kraft', 'roger', 'goodell', 'air', 'still', 'clear', 'boston', 'globe', '#google', '#sport']  ---  ['robert kraft and roger goodell air', "n't clear boston globe"]

For Robert Kraft and Roger Goodell air still isn't clear - Boston Globe http://t.co/yN68nBRFne #google #sport  ---  ['robert', 'kraft', 'roger', 'goodell', 'air', 'still', 'clear', 'boston', 'globe', '#google', '#sport']  ---  ['robert kraft and roger goodell air', "n't clear boston globe"]

Need #recipes? Visit the Kraft #CookingUpGood page &amp; watch the video for delicious ideas! http://t.co/eEkfmtgln4  ---  ['n

In [48]:
from collections import Counter

terms_all = [word for wordlist in data['cleansed_tokens'] for word in wordlist]
count_all = Counter()
# Update the counter
count_all.update(terms_all)
# Print the first 5 most frequent words
print(count_all.most_common(250))

[('kraft', 5023), ('cheese', 1374), ('goodell', 906), ('velveeta', 802), ('robert', 744), ('video', 739), ('#cookingupgood', 645), ('#recipes', 633), ('new', 522), ('need', 468), ('patriot', 434), ('nfl', 426), ('watch', 417), ('like', 403), ('idea', 394), ('delicious', 391), ('roger', 389), ('visit', 387), ('page', 387), ('year', 386), ('mac', 359), ('food', 349), ('owner', 349), ('dinner', 321), ('apology', 320), ('pocket', 303), ('walmart', 290), ('yummy', 255), ('#sweepstakes', 253), ('bob', 250), ('deflategate', 239), ('make', 222), ("i'm", 207), ('integrity', 192), ('time', 190), ('get', 176), ('relationship', 169), ('say', 166), ('investigation', 166), ('speech', 159), ('bill', 156), ('marketing', 148), ('want', 146), ('data', 145), ('#nfl', 139), ('england', 136), ('said', 135), ('harnessed', 135), ('transform', 135), ('game', 130), ('know', 126), ('center', 123), ('stand', 123), ('front', 123), ('think', 122), ('one', 121), ('news', 121), ('big', 120), ('unfairly', 120), ('att

In [52]:
terms_all_phrase = [word for wordlist in data['phrases'] for word in wordlist]
count_all = Counter()
# Update the counter
count_all.update(terms_all_phrase)
# Print the first 5 most frequent words --> meaningful phrases
print(count_all.most_common(25))

[('robert kraft', 433), ('the video', 383), ('visit the kraft', 381), ('cookingupgood page', 381), ('delicious ideas', 381), ('kraft foods', 279), ('yummy new year', 249), ('roger goodell', 212), ('kraft dinner', 125), ('bob kraft', 109), ('my pocket velveeta', 109), ('my pockets velveeta', 106), ('an apology', 94), ('nfl spotlight', 88), ('the new england patriots', 85), ('million investment', 78), ('big time', 75), ('kraft cheese', 73), ('kraft apology', 72), ('patriots owner kraft', 70), ('deflategate investigation kraft relationship nfl integrity boston herald', 62), ('caleb kraft prints modular thumbstick extentions', 59), ('my job', 57), ('the nfl', 56), ('velveeta cheese', 52)]


In [49]:
# Count terms only once, equivalent to Document Frequency
terms_single = set(terms_all)

# Count hashtags only
terms_hash = [term for term in terms_all if term.startswith('#')]

# Count usertags only
terms_usertag = [term for term in terms_all if term.startswith('@')]

# Count terms only (no hashtags, no mentions)
terms_only = [term for term in terms_all if not term.startswith(('#', '@'))] 

In [50]:
count_all = Counter()
# Update the counter
count_all.update(terms_hash)
# Print the first 5 most frequent words --> primary theme tags
print(count_all.most_common(25))

[('#cookingupgood', 645), ('#recipes', 633), ('#sweepstakes', 253), ('#nfl', 139), ('#superbowlxlix', 118), ('#kraft', 115), ('#robertkraft', 114), ('#rtmusic', 107), ('#patriots', 98), ('#deflategate', 70), ('#superbowl', 43), ('#gameday', 38), ('#gossip', 38), ('#news', 28), ('#goodell', 19), ('#coupon', 18), ('#cheese', 17), ('#rogergoodell', 16), ('#ebay', 16), ('#askroger', 15), ('#velveeta', 13), ('#sports', 12), ('#forsale', 12), ('#coupons', 11), ('#immersionjournalism', 11)]


In [51]:
count_all = Counter()
# Update the counter
count_all.update(terms_only)
# Print the first 5 most frequent words
print(count_all.most_common(25))

[('kraft', 5023), ('cheese', 1374), ('goodell', 906), ('velveeta', 802), ('robert', 744), ('video', 739), ('new', 522), ('need', 468), ('patriot', 434), ('nfl', 426), ('watch', 417), ('like', 403), ('idea', 394), ('delicious', 391), ('roger', 389), ('visit', 387), ('page', 387), ('year', 386), ('mac', 359), ('food', 349), ('owner', 349), ('dinner', 321), ('apology', 320), ('pocket', 303), ('walmart', 290)]


In [20]:
# most frequently occuring terms
countVectorizer = CountVectorizer() 
countVector = countVectorizer.fit_transform([' '.join(text) for text in data['cleansed_tokens']])
print('{} Number of tweets has {} words'.format(countVector.shape[0], countVector.shape[1]))
#print(countVectorizer.get_feature_names())

count_vect_df = pd.DataFrame(countVector.toarray(), columns=countVectorizer.get_feature_names())
count_vect_df.head()

6936 Number of tweets has 8824 words


,___blaackgirl,__mochahontas__,_alextvn,_casssshew,_dyslexic,_elllaaaa,_inamad,_jpea,_karinad,_kaylakaylee,...,zimmsy,zinc,zinga,zoomed,zootennis,zrau,zyla,ños,ôlée,öck
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# Term-Document Matrix
term_doc_df = count_vect_df.transpose()
term_doc_df.head()

,0,1,2,3,4,5,6,7,8,9,...,6926,6927,6928,6929,6930,6931,6932,6933,6934,6935
___blaackgirl,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
__mochahontas__,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
_alextvn,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
_casssshew,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
_dyslexic,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
term_doc_df.index.name='Token'

In [53]:
# Dump as excel
term_doc_df[:50].to_excel("tfidf-output.xlsx") 

In [54]:
import openpyxl 
from openpyxl.styles import Font 

#wb = openpyxl.Workbook() 
wb = openpyxl.load_workbook("tfidf-output.xlsx")
sheet = wb.active 

sheet.insert_rows(0,1)
  
# merge cell.
sheet.merge_cells('B1:Z1') 
  
sheet.cell(row = 1, column = 2).value = 'S.No.(Unique Identifier of a comment.)'

# set the font style to bold 
sheet.cell(row = 1, column = 2).font = Font(size = 24, bold = True) 
  
wb.save('tfidf-output-merge.xlsx') 

#### Summary of Exploratory Data Analysis

    - tweets are quite unclean. some taken care here. still more required to be handled such as spell checker, more accurate phrase extracter using rule based parser or using NER..etc.
    - Most of the short cuts or abbreviations can either be transformed or dropped. Needs to be taken care after Subject knowledge.
    - used lemmatizer, in this dataset.
    - due to lack of memory(3GB RAM) in my system, i'm dumping few samples of term-frequency records to excel. user can use same code to dump all records with having good memory system. or we can do other ways like using DASK, or writing file batch by batch.
    - have mentioned comments with each code. i'm sure those are understandable. so not writing much here.
    - with that 4 questions regarding to this task have been answered through code, comments, and image of results.

In [10]:
#import itertools
#from nltk import ngrams
from pattern.en import parsetree

def get_phrases(sentence):
    phrase_dict = {}
    parse_tree = parsetree(sentence, relations=True, lemmata=True)
    for sent in parse_tree:
        for chunk in sent.chunks:
            phrase_key = chunk.type
            phrase_val = ' '.join([w.string for w in chunk.words])

            if not phrase_dict.get(phrase_key, []):
                phrase_dict[phrase_key] = []
            if len(phrase_val.split())>1:
                phrase_dict[phrase_key].append(phrase_val)

    return phrase_dict

if __name__ == "__main__":
    text = """outlook application is not working on my spam email"""
    print(get_phrases(text))
    text = """RT @kittyFitz50: I love Kraft beer .@America_On_Tap - Madison Ads on Limbaugh tarnish your image. #StopRush"""
    print(get_phrases(text))
    text = """check this out For Robert Kraft and Roger Goodell air still isn't clear - Boston Globe: Bosto... http://t.co/fXF5wfilgt #sports #online"""
    print(get_phrases(text))
    text = """RT @OTLonESPN: Tomorrow 8am ET ESPN2: Fallout from "Deflategate"reaction to Friday’s comments from Goodell"""
    print(get_phrases(text))
    text = """@EdgeofSports You take the good-You take the bad-Givin Bob Kraft hand jobs ain't so bad-The Facts of Life-The Facts of Life#tootiebowl"""
    print(get_phrases(text))
    text = """For Robert Kraft and Roger Goodell air still isn't clear - Boston Globe http://t.co/Rw4oh3hswK #google #sport"""
    print(get_phrases(text))



{'NP': ['outlook application', 'my spam email'], 'VP': ['is not working'], 'PP': []}
{'NP': ['Kraft beer', 'Madison Ads', 'your image'], 'VP': [], 'PP': []}
{'NP': ['Robert Kraft and Roger Goodell air', 'Boston Globe'], 'ADVP': [], 'PP': [], 'VP': ['still is'], 'ADJP': ["n't clear"]}
{'NP': ['Tomorrow 8am ET ESPN2', 's comments'], 'VP': [], 'PP': []}
{'NP': ['@EdgeofSports You', 'the good-You', 'the bad-Givin Bob Kraft hand jobs', 'Life-The Facts'], 'VP': [], 'ADVP': ["n't so"], 'PP': []}
{'PP': [], 'NP': ['Robert Kraft and Roger Goodell air', 'Boston Globe http://t.co/Rw4oh3hswK'], 'VP': ['still is'], 'ADJP': ["n't clear"]}


In [126]:
#! pip install pattern

In [7]:
import os 
import re 
import sys 
import string
import codecs 
import subprocess
from pattern.en import parse

In [9]:
import os 
import re 
import sys 
import string
import codecs 
import subprocess

#filepath = os.path.abspath(os.path.dirname(__file__))

#reload(sys)  
#sys.setdefaultencoding('utf8')

from pattern.en import parse

stopwords = [u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u'not', u'only', u'own', u'same', u'so', u'than', u'too', u'very', u's', u't', u'can', u'will', u'just', u'don', u'should', u'now']



### START Keyword extraction ### 

def extract_link(text):
	regex = r'https?://[^\s<>"]+|www\.[^\s<>)"]+'
	match = re.findall(regex, text)
	links = [] 
	for x in match: 
		if x[-1] in string.punctuation: links.append(x[:-1])
		else: links.append(x)
	return links
	
def cleanup(query): 
	try:
		urls = extract_link(" " + query + " ")
		for url in urls: 
			query = re.sub(url, "", query)
		q = query.strip()
	except:
		q = query
	q = re.sub(' RT ', '', ' ' + q + ' ').strip() 
	return q 


def convert_tag_format(query): 
	word = query.split(' ')
	postag = [(x.split('/')[0], x.split('/')[1]) for x in word]
	return postag 
	

def get_pos_tags(text): 
	tagged_sent = parse(text)	
	return convert_tag_format(tagged_sent), tagged_sent
	
def normalise(word):
	word = word.lower()
	return word


## conditions for acceptable word: length, stopword
def acceptable_word(word):
    accepted = bool(2 <= len(word) <= 40
        and word.lower() not in stopwords)
    return accepted

## extract entity from BIO encoding 
def extract_entity(filetext):
	last_entity = '' 
	last_tag = '' 
	mention2entities = {} 
	for line in filetext.split('\n'): 
		line = line.strip() 
		if line == '': 
			continue
		line_split = line.split('\t')
		if re.search('B-', line_split[1]): 
			if last_entity != '': 
				if not last_tag in mention2entities:
					mention2entities[last_tag] = [] 
				mention2entities[last_tag].append(last_entity.strip())
			last_entity = line_split[0] + ' '
			last_tag = line_split[1][2:] 
		elif re.search('I-', line_split[1]): 
			last_entity += line_split[0] + ' '
	if last_entity != '': 
		if not last_tag in mention2entities:
			mention2entities[last_tag] = [] 
		mention2entities[last_tag].append(last_entity.strip())
	return 	mention2entities

	
def get_entities_from_phrase(tagged_sent, phrase2consider): 
	word = tagged_sent.split(' ')
	bio_tags = [normalise(x.split('/')[0])+ '\t'+ x.split('/')[2] for x in word]
	bio_text = '\n'.join(bio_tags)
	mention2entities = extract_entity(bio_text)
	print(mention2entities.keys())
	
	## strip off unacceptable words 
	_mention2entities = {} 
	for mention in mention2entities: 
		if not mention in phrase2consider: 
			continue
		_mention2entities[mention] = [] 
		for entity in mention2entities[mention]: 
			_entity = ' '.join([word for word in entity.split(' ') if acceptable_word(word)]).strip()
			if _entity != '': 
				_mention2entities[mention].append(_entity)
			
	entities = []
	for mention in _mention2entities: 
		entities.extend(_mention2entities[mention])
	return entities	
	

def get_keywords(text, phrase2consider=['NP', 'ADJP']): 
	_text = cleanup(text)
	try:
		postoks, tagged_sent = get_pos_tags(_text)
		entities = get_entities_from_phrase(tagged_sent, phrase2consider)
	except: 
		return []
	return entities

### END Keyword extraction ### 

### START other entity extraction ### 
def extract_hashtag(text, to_normalize=True):
	regex = r'#[^\W\d_]+\b'
	if to_normalize: text = normalise(text)
	match = re.findall(regex, text)
	return match

def extract_users(text, to_normalize=True):
#	regex = r'@[^\W\d_]+\b'
	regex = r'@[^\b ]+\b'
	if to_normalize: text = normalise(text)
	match = re.findall(regex, text)
	return match


def get_emoji_list():
	emoji_text = codecs.open(filepath+'/emoji_table.txt', 'r', 'utf-8').read().split('\n')
	emojis = [x.split(',')[0] for x in emoji_text][1:]
	emojis = [x for x in emojis if x.strip() != '']
	return emojis
#emojis = get_emoji_list()
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

#Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

#combine sad and happy emoticons
emojis = emoticons_happy.union(emoticons_sad)

def extract_emojis(text): 
	global emojis
	_emos = [] 
	for each in emojis: 
		each = each.strip() 
		if each == '': continue
		if each in text: 
			_emos.extend(re.findall(each, text.decode('utf-8')))
	
#	_emos2 = []
#	for char in text.decode('utf-8'): 
#		if char in _emos: 
#			_emos2.append(char)		
	return _emos#, _emos2 
	
### END other entity extraction ### 

def all_entities(text, to_normalize=True, with_unigram=True): 
	text = text.decode('utf-8')
	if to_normalize: text = normalise(text)
	if with_unigram: 
		entities = text.split()
	else: 
		entities = [] 
	keywords = get_keywords(text, ['NP', 'VP', 'ADJP', 'ADVP'])
	emojis = extract_emojis(text)
	for each in keywords+emojis: 
		if not each in entities:
			entities.append(each)
	return entities
	
if __name__ == '__main__':	
	queries = ["The mobile web is more important than mobile apps.", "As a #roadmapscholar, I highly recommend #startup bootcamp for #founders by @andrewsroadmaps : http://t.co/ZBISIMEBRH (http://t.co/VF5CojRWNF)", "RT @andrewsroadmaps: Proud of @naushadzaman &amp; @WasimKhal for winning the #IBMWatson hackathon! #roadmapscholars https://t.co/08sbAjKWKu."]
	queries = ["""outlook application is not working on my spam email""","""RT @kittyFitz50: I love Kraft beer .@America_On_Tap - Madison Ads on Limbaugh tarnish your image. #StopRush""","""check this out For Robert Kraft and Roger Goodell air still isn't clear - Boston Globe: Bosto... http://t.co/fXF5wfilgt #sports #online""","""RT @OTLonESPN: Tomorrow 8am ET ESPN2: Fallout from "Deflategate"reaction to Friday’s comments from Goodell""","""@EdgeofSports You take the good-You take the bad-Givin Bob Kraft hand jobs ain't so bad-The Facts of Life-The Facts of Life#tootiebowl""","""For Robert Kraft and Roger Goodell air still isn't clear - Boston Globe http://t.co/Rw4oh3hswK #google #sport"""]
    
	for query in queries: 
		print('query', query)
		print('get_keywords(query)', get_keywords(query))
		print("get_keywords(query, ['NP'])", get_keywords(query, ['NP']))
		print("extract_hashtag(query)", extract_hashtag(query)) 
		print("extract_users(query)", extract_users(query))
		#print "extract_link(query)", extract_link(query)
		print()


query outlook application is not working on my spam email
dict_keys(['NP', 'VP', 'PP'])
get_keywords(query) ['outlook application', 'spam email']
dict_keys(['NP', 'VP', 'PP'])
get_keywords(query, ['NP']) ['outlook application', 'spam email']
extract_hashtag(query) []
extract_users(query) []

query RT @kittyFitz50: I love Kraft beer .@America_On_Tap - Madison Ads on Limbaugh tarnish your image. #StopRush
dict_keys(['NP', 'VP', 'PP'])
get_keywords(query) ['@kittyfitz50', 'kraft beer', 'madison ads', 'limbaugh', 'image', 'stoprush']
dict_keys(['NP', 'VP', 'PP'])
get_keywords(query, ['NP']) ['@kittyfitz50', 'kraft beer', 'madison ads', 'limbaugh', 'image', 'stoprush']
extract_hashtag(query) ['#stoprush']
extract_users(query) ['@kittyfitz50', '@america_on_tap']

query check this out For Robert Kraft and Roger Goodell air still isn't clear - Boston Globe: Bosto... http://t.co/fXF5wfilgt #sports #online
dict_keys(['NP', 'ADVP', 'PP', 'VP', 'ADJP'])
get_keywords(query) ['check', 'robert kraft 

In [178]:
! pip3 install openpyxl
#! pip3 install pattern

  Created wheel for et-xmlfile: filename=et_xmlfile-1.0.1-py3-none-any.whl size=8920 sha256=715dd79b729c4871bee51760b0ed100ed9feedaafb6e71ab5f28845b2c736f9b
  Stored in directory: c:\users\achyuta\appdata\local\pip\cache\wheels\e2\bd\55\048b4fd505716c4c298f42ee02dffd9496bb6d212b266c7f31
Successfully built et-xmlfile
